In [1]:
import os
import shutil
import cv2
import glob
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
original_datapath = r"C:\Users\dgyawa1\Desktop\dev\food_data"
training_datapath = r"C:\Users\dgyawa1\Desktop\dev\yolo_data"
annotation_dir = r"C:\Users\dgyawa1\Desktop\dev\annot_data"

In [11]:
if not os.path.isdir(training_datapath):
        os.makedirs(training_datapath)

i = 0
for label in os.listdir(original_datapath):
    label_path = os.path.join(original_datapath,label)
    for img in os.listdir(label_path):
        src_path = os.path.join(label_path,img)
        if img.endswith(".jpg"):
            dest_filename = os.path.splitext(img)[0] + "_" + label +".jpg"
            dest_path = os.path.join(training_datapath,dest_filename)
            shutil.copy(src_path,dest_path)
        if img.endswith(".txt"):
            with open(src_path,"r") as f:
                for line in f.readlines():
                    all_line = line.split(" ") 
                    all_line_copy = all_line.copy()
                    txt_filename = all_line[0] + "_" + label +".txt"
                    img_arr = cv2.imread(os.path.join(label_path,all_line[0]+'.jpg'))
                    height,width,_ = img_arr.shape
                    all_line[0] = str(i)
                    all_line[1] = str(((int(all_line_copy[1]) + int(all_line_copy[3]))/2)/width)
                    all_line[2] = str(((int(all_line_copy[2]) + int(all_line_copy[4]))/2)/height)
                    all_line[3] = str((int(all_line_copy[3]) - int(all_line_copy[1]))/width)
                    all_line[4] = str((int(all_line_copy[4]) - int(all_line_copy[2]))/width)
                    txt_destination = os.path.join(training_datapath,txt_filename)
                    with open(txt_destination, "w") as fname:
                        fname.write(" ".join(all_line))
    i = i+1

## Visualize the dataset

In [41]:
all_images = glob.glob(r"C:\Users\dgyawa1\Desktop\dev\yolo_data\*.jpg")

label_dict = {
    "0":"chicken",
    "1": "fries",
    "2": "lettuce",
    "3": "pizza",
    "4": "roll"
}


for file in tqdm(all_images):
    img_name = os.path.splitext(os.path.basename(file))[0]
    file_name = os.path.join(training_datapath,img_name+".txt")
    with open(file_name,"r") as f:
        for i in f.readlines():
            all_line = i.split(" ")
            label,x0,y0,x2,y2 = all_line[0], int(all_line[1]), int(all_line[2]), int(all_line[3]), int(all_line[4])
            img = cv2.imread(file)
#             img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            cv2.rectangle(img,(x0,y0),(x2,y2),(0,0,255),2)
            cv2.putText(img,label_dict[label],(x0+5,y0-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2)
            save_dir = os.path.join(annotation_dir,os.path.basename(file))
            cv2.imwrite(save_dir,img)    

100%|██████████| 890/890 [00:10<00:00, 88.42it/s] 


## Convert segmentation mask to bounding box

In [45]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F


ASSETS_DIRECTORY = "assets"

plt.rcParams["savefig.bbox"] = "tight"


def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

C:\Users\dgyawa1\Desktop\dev\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] The specified module could not be found
  warn(f"Failed to load image Python extension: {e}")


In [52]:
from torchvision.io import read_image

mask_path = r"C:\Users\dgyawa1\Downloads\FoodSeg103\FoodSeg103\Images\ann_dir\train\00000000.jpg"
img_path = r"C:\Users\dgyawa1\Downloads\FoodSeg103\FoodSeg103\Images\img_dir\train\00000000.jpg"

img = read_image(img_path)
mask = read_image(mask_path)

print(mask.size())
print(img.size())
print(mask)

RuntimeError: No such operator image::read_file